# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [191]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
 


# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [192]:
#setting file path to read in
file = os.path.join("..", "Output_Data", "cities.csv")

#read in csv
city_weather = pd.read_csv(file, delimiter=',')
city_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Westport,41.1415,-73.3579,35.01,100,82,7.02,US,1609910785
1,Albany,42.6001,-73.9662,30.00,85,88,4.81,US,1609910787
2,Narsaq,60.9167,-46.0500,21.20,44,28,5.82,GL,1609910788
3,Baykal'sk,51.5058,104.1469,3.07,88,100,11.45,RU,1609910789
4,Leningradskiy,69.3833,178.4167,-9.85,88,100,9.22,RU,1609910790


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [193]:
gmaps.configure(api_key=gkey)

In [195]:
#set map locations and weight 
locations = city_weather[["Lat", "Lng"]].astype(float)
humidity = city_weather["Humidity"].astype(float)

#generate gmap 
fig = gmaps.figure(center=(51.3, 0), zoom_level=1.3)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

#add heat layer to gmap 
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [196]:
#narrowing down cities based on max temp, wind speed and cloudiness conditions
ideal_weather = city_weather.loc[(city_weather["Max Temp"] >= 73) & (city_weather["Max Temp"] <= 80) 
                                  & (city_weather["Wind Speed"] < 10) & (city_weather["Cloudiness"] == 0)]
ideal_weather.dropna()
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
241,Praia,14.9215,-23.5087,73.4,49,0,9.17,CV,1609911089
310,Castanhal,-1.2939,-47.9264,77.0,88,0,6.93,BR,1609911176
490,Praia,14.9215,-23.5087,73.4,49,0,9.17,CV,1609911409
507,Nicoya,10.1483,-85.4520,75.2,73,0,3.36,CR,1609911261
517,Santarém,-2.4431,-54.7083,73.4,100,0,3.36,BR,1609911445


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [197]:
#rename ideal weather df
hotel_df = pd.DataFrame(data=ideal_weather)
hotel_df["Hotel Name"] = ""
hotel_df.reindex()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
241,Praia,14.9215,-23.5087,73.4,49,0,9.17,CV,1609911089,
310,Castanhal,-1.2939,-47.9264,77.0,88,0,6.93,BR,1609911176,
490,Praia,14.9215,-23.5087,73.4,49,0,9.17,CV,1609911409,
507,Nicoya,10.1483,-85.4520,75.2,73,0,3.36,CR,1609911261,
517,Santarém,-2.4431,-54.7083,73.4,100,0,3.36,BR,1609911445,


In [198]:
#create empty list to store hotels
hotels = []

#set call parameters 
params = {"keyword": "Hotel",
          "radius": 5000,
          "type": "lodging",
          "key": gkey}

#iterate through json rows to add first hotel to df
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    response_json = response.json()
    #print(json.dumps(response_json, indent=4, sort_keys=True))
    
    try:
        hotels.append(response_json["results"][0]["name"])
    
    except (IndexError):
        hotels.append(np.nan)

hotel_df["Hotel Name"] = hotels
hotel_df.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
241,Praia,14.9215,-23.5087,73.4,49,0,9.17,CV,1609911089,Hotel Oasis Atlantico Praiamar
310,Castanhal,-1.2939,-47.9264,77.0,88,0,6.93,BR,1609911176,Accord Hotel
490,Praia,14.9215,-23.5087,73.4,49,0,9.17,CV,1609911409,Hotel Oasis Atlantico Praiamar
507,Nicoya,10.1483,-85.4520,75.2,73,0,3.36,CR,1609911261,Hotel Doña Marta
517,Santarém,-2.4431,-54.7083,73.4,100,0,3.36,BR,1609911445,Barrudada Tropical Hotel


In [189]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [190]:
#Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))